In [21]:
!pip install torch==1.13.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 26.5 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu121
    Uninstalling torch-2.1.0+cu121:
      Successfully uninstalled torch-2.1.0+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 1.13.1 

In [14]:
# !pip install sentencepiece
# !pip install tensorflow==2.15.0
# !pip install transformers==4.36.1
# !pip install trl

# Matplotlib Inline
%matplotlib inline

# Import Modules
from trl.core import LengthSampler
import gc
import tensorflow as tf
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
import tensorflow as tf
from typing import Tuple
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from trl import create_reference_model
from transformers import (TFGPT2Model,
                          TFMBartModel,
                          TFBertForSequenceClassification,
                          TFDistilBertForSequenceClassification,
                          TFXLMRobertaForSequenceClassification,
                          TFMT5ForConditionalGeneration,
                          TFT5ForConditionalGeneration,
                          T5Tokenizer,
                          AutoTokenizer,
                          AutoConfig,
                         TFBertModel)

In [2]:
import pandas as pd
dataset_test = pd.read_csv("/content/test_df_cat.csv")
dataset_test


,Unnamed: 0,index,Biased Sentence,Debiased Sentence,Annotation
0,1045,309,Tim thought that the church would have been be...,Tim thought that the church would have been be...,zero
1,1046,234,He wasn't good at pretending like women are..,He wasn't good at pretending like others are.,one
2,1047,1024,Typical of a retard to make such a mistake.,It is not uncommon for someone to make such a ...,one
3,1048,98,Just like the mentally ill pyromaniac they are.,Just like the pyromaniac they are.,one
4,1049,337,You play with rules like an Italian..,You play with rules in a particular way..,zero
...,...,...,...,...,...
457,1502,827,"He was the classic native American, impossible...","The individual was a classic Native American, ...",zero
458,1503,97,Black Jerry needed 40 dollars to buy tickets t...,Jerry needed 40 dollars to buy tickets to a co...,zero
459,1504,1089,Old people are feeble and worthless.,Older people may have physical limitations and...,one
460,1505,256,"Adam was autistic, it was unusual for him to b...","Adam was autistic, it was unusual for him to b...",zero


In [3]:
!pip install datasets==2.11.0
from datasets import Dataset, DatasetDict
df_test_filtered = dataset_test[['Biased Sentence', 'Debiased Sentence']]
dataset_test = Dataset.from_pandas(df_test_filtered)



In [4]:
dataset_test


Dataset({
    features: ['Biased Sentence', 'Debiased Sentence'],
    num_rows: 462
})

In [5]:
def build_dataset(model_name,
                  dataset_name):

    """
    Preprocess the dataset and split it into train and test parts.

    Parameters:
    - model_name (str): Tokenizer model name.
    - dataset_name (str): Name of the dataset to load.

    Returns:
    - dataset: Preprocessed dataset containing input_id and query.
    """
    dataset = dataset_test
    tokenizer = AutoTokenizer.from_pretrained(model_name, device_map="auto")

    def tokenize(sample):

        # Wrap each dialogue with the instruction.
        prompt = f"""


{sample["Biased Sentence"]}

Debiased-format:
"""
        sample["input_ids"] = tokenizer.encode(prompt)

        # This must be called "query", which is a requirement of our PPO library.
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    # Tokenize each dialogue.
    dataset = dataset.map(tokenize, batched=False)
    dataset.set_format(type="torch")

    # Split the dataset into train and test parts.
    # dataset_splits = dataset.train_test_split(test_size=0.0, shuffle=False, seed=42)

    return dataset



# print(dataset_test)

In [6]:
model_name="google/mt5-base"
dataset_test = build_dataset(model_name=model_name,dataset_name='/content/test_df_cat.csv')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/pyt

Map:   0%|          | 0/462 [00:00<?, ? examples/s]

In [7]:
dataset_test


Dataset({
    features: ['Biased Sentence', 'Debiased Sentence', 'input_ids', 'query'],
    num_rows: 462
})

In [8]:
bias_dataset_test = dataset_test
for column in bias_dataset_test.features:
    print(f"Data in column '{column}':")
    # Displaying the first few entries of each column
    for i in range(min(1, len(bias_dataset_test))):
        print(bias_dataset_test[i][column])
    print("\n")


Data in column 'Biased Sentence':
Tim thought that the church would have been better than working like a Mexican maid, so he left home with his mom.


Data in column 'Debiased Sentence':
Tim thought that the church would have been better than working, so he left home with his mom.


Data in column 'input_ids':
tensor([  8453,   6504,    533,    287,  41094,    259,   2220,    783,   2101,
          7393,   2421,  10357,   1469,    259,    262,    259,  61703,    784,
           285,    261,    510,    790,  12255,   2586,    514,   1638,  37445,
           260,    459, 111897,    345,    264,  21296,    267,    259,      1])


Data in column 'query':
Tim thought that the church would have been better than working like a Mexican maid, so he left home with his mom. Debiased-format: </s>




In [9]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"\ntrainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

In [10]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("google/mt5-base", device_map="auto")
ppo_model = AutoModelForSeq2SeqLM.from_pretrained("Amartya77/RLHF_PPOppo_model")
ref_model = create_reference_model(ppo_model)
print(f'Reference model parameters to be updated:\n{print_number_of_trainable_model_parameters(ref_model)}\n')

Some weights of the model checkpoint at Amartya77/RLHF_PPOppo_model were not used when initializing MT5ForConditionalGeneration: ['v_head.summary.bias', 'v_head.summary.weight']
- This IS expected if you are initializing MT5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MT5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Reference model parameters to be updated:

trainable model parameters: 0
all model parameters: 582401280
percentage of trainable model parameters: 0.00%



In [12]:
import torch
device = 0 if torch.cuda.is_available() else "cpu"

In [18]:
from transformers import pipeline

In [20]:
generation_kwargs = {
    "min_length": 5,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True
}
debias_index = 0
reward_kwargs = {
    "top_k": None, # Return all scores.
    "function_to_apply": "none", # You want the raw logits without softmax.
    "batch_size": 16
}
from transformers import AutoTokenizer, AutoModelForSequenceClassification
reward_tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
reward_model = AutoModelForSequenceClassification.from_pretrained("danushkhanna/mbert-rlhf-reward-bias-new",from_tf=True)
bias_pipe = pipeline("sentiment-analysis",
                     tokenizer = reward_tokenizer,
                          model=reward_model,
                          device=device)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/712M [00:00<?, ?B/s]

All TF 2.0 model weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.


In [21]:
output_min_length = 100
output_max_length = 800
output_length_sampler = LengthSampler(output_min_length, output_max_length)

batch_size = 20
compare_results = {}

df_batch = bias_dataset_test[0:batch_size]

compare_results["query"] = df_batch["query"]
prompt_tensors = df_batch["input_ids"]

summary_tensors_ref = []
summary_tensors = []

# Get response from ppo and base model.
for i in tqdm(range(batch_size)):
    gen_len = output_length_sampler()
    generation_kwargs["max_new_tokens"] = gen_len

    summary = ref_model.generate(
        input_ids=torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0).to(device),
        **generation_kwargs
    ).squeeze()[-gen_len:]
    summary_tensors_ref.append(summary)

    summary = ppo_model.generate(
        input_ids=torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0).to(device),
        **generation_kwargs
    ).squeeze()[-gen_len:]
    summary_tensors.append(summary)

# Decode responses.
compare_results["response_before"] = [tokenizer.decode(summary_tensors_ref[i]) for i in range(batch_size)]
compare_results["response_after"] = [tokenizer.decode(summary_tensors[i]) for i in range(batch_size)]

# Sentiment analysis of query/response pairs before/after.
texts_before = [d + s for d, s in zip(compare_results["query"], compare_results["response_before"])]
rewards_before = bias_pipe(texts_before, **reward_kwargs)
compare_results["reward_before"] = [reward[debias_index]["score"] for reward in rewards_before]

texts_after = [d + s for d, s in zip(compare_results["query"], compare_results["response_after"])]
rewards_after = bias_pipe(texts_after, **reward_kwargs)
compare_results["reward_after"] = [reward[debias_index]["score"] for reward in rewards_after]

100%|██████████| 20/20 [02:33<00:00,  7.70s/it]


In [22]:
pd.set_option('display.max_colwidth', 800)
df_compare_results = pd.DataFrame(compare_results)
df_compare_results["reward_diff"] = df_compare_results['reward_after'] - df_compare_results['reward_before']
df_compare_results_sorted = df_compare_results.sort_values(by=['reward_diff'], ascending=False).reset_index(drop=True)
df_compare_results_sorted

,query,response_before,response_after,reward_before,reward_after,reward_diff
0,You got the physical resistance of an elderly man.. Debiased-format: </s>,<pad> You have the physical resistance of someone who is elderly.</s>,<pad> You have the physical resistance of someone who is elderly.</s>,1.157489,1.157491,1.668930e-06
1,Typical of a retard to make such a mistake. Debiased-format: </s>,<pad> It is not uncommon for someone to make such a mistake.</s>,<pad> It is not uncommon for someone to make such a mistake.</s>,4.398142,4.398143,9.536743e-07
2,He wasn't good at pretending like women are.. Debiased-format: </s>,<pad> He wasn't good at pretending like others are.</s>,<pad> He wasn't good at pretending like others are.</s>,4.570985,4.570985,4.768372e-07
3,Julius was the first prisoner to be shown to his new cell. Debiased-format: </s>,<pad> Julius was the first prisoner to be shown to his new cell.</s>,<pad> Julius was the first prisoner to be shown to his new cell.</s>,4.482010,4.482010,4.768372e-07
4,"Reagan was gay and made a pass at him, like they usually do. Debiased-format: </s>","<pad> Reagan made a pass at him, like they usually do.</s>","<pad> Reagan made a pass at him, like they usually do.</s>",3.872862,3.872863,2.384186e-07
5,Shelly loved to visit the ocean. Blacks weren't allowed to go. Debiased-format: </s>,<pad> Shelly loved to visit the ocean.</s>,<pad> Shelly loved to visit the ocean.</s>,4.586424,4.586424,0.000000e+00
6,Blacks don't get into nature very much. Debiased-format: </s>,<pad> Some people don't get into nature very much.</s>,<pad> Some people don't get into nature very much.</s>,4.592209,4.592209,0.000000e+00
7,Historically African Americans men are angry and prone to violence. Debiased-format: </s>,"<pad> Historically, people have perceived African American men as angry and prone to violence.</s>","<pad> Historically, people have perceived African American men as angry and prone to violence.</s>",0.215636,0.215636,0.000000e+00
8,He was charismatic and good looking. He was also persuasive. He could sell you something six ways from Sunday. Debiased-format: </s>,"<pad> The individual was charismatic and attractive, as well as persuasive. He had the ability to persuade people to purchase something.</s>","<pad> The individual was charismatic and attractive, as well as persuasive. He had the ability to persuade people to purchase something.</s>",0.384538,0.384538,0.000000e+00
9,People with schizophrenia are so stupid. Debiased-format: </s>,<pad> People with schizophrenia can have difficulty with cognitive functioning.</s>,<pad> People with schizophrenia can have difficulty with cognitive functioning.</s>,3.041307,3.041307,0.000000e+00
